# Modelización
En este _notebook_ vamos a modelizar los mismos datos que en R usando también XGBoost (y _gridsearch_).

## Modelización con árboles

En primer lugar construiremos un árbol.

In [ ]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn import preprocessing

Leemos los datos como antes.

In [ ]:
rents = pd.read_table("../data/rent.txt", encoding = 'latin_1')

Preparamos los datos eliminando la variable objetivo que no nos interesa.

In [ ]:
tmp = rents.copy()
tmp = tmp.drop('Rent per sqm', axis = 1)
#tmp.columns

Seleccionamos las variables objetivo y predictoras.

In [ ]:
Y = tmp[['Rent']].copy()
X = tmp.drop('Rent', axis = 1).copy()

Desafortunadamente, algunas de esas variables no son numéricas y en Python es obligatorio preprocesarlas.

In [ ]:
X.dtypes

Vamos a codificarlas como _dummies_, es decir, usando 0/1 para indicar ausencia/presencia de un nivel.

In [ ]:
df = rents.select_dtypes(include=['object']).copy()
X = pd.get_dummies(X, columns = df.columns, drop_first = True)
X.head()

In [ ]:
X.dtypes

Ajustamos el árbol y lo exportamos para poder inspeccionarlo.

In [ ]:
modelo = tree.DecisionTreeRegressor(max_depth = 4)
modelo = modelo.fit(X, Y)

In [ ]:
tree.export_graphviz(modelo, 
                     feature_names = X.columns,
                     out_file = '/tmp/tree.dot')  

Ahora, haciendo

```
cd /tmp
dot -Tpng tree.dot -o tree.png
xdg-open tree.png
```

se puede crear e inspeccionar el fichero `.png` que describe el árbol.

#### Ejercicio:
Consultando la [documentación de `DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html), crea otros árboles usando diferentes opciones.

Se pueden obtener las predicciones así:

In [ ]:
preds = modelo.predict(X)

#### Ejercicio

Crea otro modelo usando el precio por metro cuadrado como variable objetivo. Luego calcula las predicciones y mira a ver cuál yerra menos en promedio.

#### Ejercicio

Usa `train_test_split` para partir el conjunto de datos en conjuntos de entrenamiento (70%) y validación (30%). Recrea ambos modelos usando la parte de _train_ y mide el error en _test_. Compara ambos modelos (usando como variables objetivo `Rent` o `Rent per sqm`).

## Modelización con XGBoost


En primer lugar, cargamos los paqutes necesarios. Puede que necesites escribir antes
```
pip install xgboost
```
si es que el paquete `xgboost` no está instalado en Anaconda.

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance

Ahora creamos una rejilla de búsqueda de parámetros

In [ ]:
my_xgb = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'eval_metric': ['rmse'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(my_xgb,
                        parameters,
                        cv = 5,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X, Y)

Vamos a mostar el error del mejor modelo:

In [ ]:
print(xgb_grid.best_score_)

Y los parámetros con los cuales se la logrado dicho valor mínimo.

In [ ]:
print(xgb_grid.best_params_)

Podemos crear predicciones:

In [ ]:
preds_xgboost = xgb_grid.predict(X)

Indicadores de la importancia de cada predictor:

In [ ]:
importancias = pd.DataFrame({'cols': X.columns, 'importancias': xgb_grid.best_estimator_.feature_importances_})
importancias.sort_values(by = 'importancias', ascending = False, inplace = True)
importancias

In [ ]:
%matplotlib inline
#import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 12

plot_importance(xgb_grid.best_estimator_)